In [2]:
## Import Packages
from __future__ import print_function

import numpy as np
import pandas as pd
from itertools import product

#Astro Software
import astropy.units as units
from astropy.coordinates import SkyCoord
from astropy.io import fits

#Plotting Packages
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import rcParams

import seaborn as sns

from PIL import Image

from yt.config import ytcfg
import yt
import yt.units as u

#Scattering NN
import torch
import torch.nn.functional as F
from torch import optim
from kymatio.torch import Scattering2D
device = "cpu"

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.decomposition import PCA, FastICA

import skimage
from skimage import filters

from scipy.optimize import curve_fit
from scipy import linalg
from scipy import stats
from scipy.signal import general_gaussian

#I/O
import h5py
import pickle
import glob
import copy
import time

#Plotting Style
%matplotlib inline
#plt.style.use('dark_background')
rcParams['text.usetex'] = False
rcParams['axes.titlesize'] = 20
rcParams['xtick.labelsize'] = 16
rcParams['ytick.labelsize'] = 16
rcParams['legend.fontsize'] = 12
rcParams['axes.labelsize'] = 20
rcParams['font.family'] = 'sans-serif'

#Threading
torch.set_num_threads=2
from multiprocessing import Pool

import ntpath
def path_leaf(path):
    head, tail = ntpath.split(path)
    out = os.path.splitext(tail)[0]
    return out

def hd5_open(file_name,name):
    f=h5py.File(file_name,'r', swmr=True)
    data = f[name][:]
    f.close()
    return data

from matplotlib.colors import LinearSegmentedColormap
cdict1 = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 0.0, 0.0),
                   (1.0, 0.0, 0.0)),

         'blue':  ((0.0, 0.0, 1.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 0.0, 0.0))
        }
blue_red1 = LinearSegmentedColormap('BlueRed1', cdict1,N=5000)

from sklearn.preprocessing import StandardScaler

/opt/miniconda3/envs/IWST/lib/python3.8/_collections_abc.py:666: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  self[key]


In [2]:
filter_bank2D = hd5_open('../scratch_AKS/data/filter_bank_paper_export.h5','main/data')
filter_bank2D_J_L = hd5_open('../scratch_AKS/data/filter_bank_paper_export.h5','main/J_L')
filter_bank2D_psi_index = hd5_open('../scratch_AKS/data/filter_bank_paper_export.h5','main/psi_index')
filter_bank2D_phi_index = hd5_open('../scratch_AKS/data/filter_bank_paper_export.h5','main/phi_index')

In [3]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/filter_bank.h5", "w")
hf.create_dataset("data", data=filter_bank2D, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("J_L", data=filter_bank2D_J_L, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("psi_index", data=filter_bank2D_psi_index, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("phi_index", data=filter_bank2D_phi_index, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [4]:
rod_corner = hd5_open('../scratch_AKS/paper_data/corner_rod.h5','main/coeffs')
rod_corner_lab = hd5_open('../scratch_AKS/paper_data/corner_rod.h5','main/labels')
rod_ex_images = hd5_open('../scratch_AKS/paper_data/images.h5','main/data')

rod_curve = hd5_open('../scratch_AKS/paper_data/curve_rod_40.h5','main/coeffs')
rod_curve_lab = hd5_open('../scratch_AKS/paper_data/curve_rod_40.h5','main/labels')
rod_ex_curves = hd5_open('../scratch_AKS/paper_data/curves.h5','main/data')

disc_data = hd5_open('../scratch_AKS/paper_data/disc_data_log.h5','main/coeffs')
disc_data_lab = hd5_open('../scratch_AKS/paper_data/disc_data_log.h5','main/labels')
disc_ex_images = hd5_open('../scratch_AKS/paper_data/discs.h5','main/data')

In [5]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/rod_space.h5", "w")
hf.create_dataset("coeffs", data=rod_corner, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("labels", data=rod_corner_lab, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("examples", data=rod_ex_images, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [6]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/curve_space.h5", "w")
hf.create_dataset("coeffs", data=rod_curve, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("labels", data=rod_curve_lab, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("examples", data=rod_ex_curves, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [7]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/disc_space.h5", "w")
hf.create_dataset("coeffs", data=disc_data, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("labels", data=disc_data_lab, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("examples", data=disc_ex_images, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [8]:
mnist_DHC_out_angles = hd5_open('../from_cannon/2021_01_20/mnist_DHC_test_angles.h5','main/data')

In [9]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/mnist_eqws_test_angles.h5", "w")
hf.create_dataset("coeffs", data=mnist_DHC_out_angles, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [10]:
mnist_DHC_out_angles = hd5_open("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/mnist_eqws_test_angles.h5",'coeffs')

In [11]:
mnist_DHC_out_rotpi_3 = hd5_open('../scratch_AKS/data/mnist_DHC_out_rotpi_3.h5','main/data')
mnist_DHC_out_rotpi_3_test = hd5_open('../scratch_AKS/data/mnist_DHC_out_rotpi_3_test.h5','main/data')

mnist_DHC_out_rot2pi_3 = hd5_open('../scratch_AKS/data/mnist_DHC_out_rot2pi_3.h5','main/data')
mnist_DHC_out_rot2pi_3_test = hd5_open('../scratch_AKS/data/mnist_DHC_out_rot2pi_3_test.h5','main/data')

mnist_DHC_out_rot3pi_3 = hd5_open('../scratch_AKS/data/mnist_DHC_out_rot3pi_3.h5','main/data')
mnist_DHC_out_rot3pi_3_test = hd5_open('../scratch_AKS/data/mnist_DHC_out_rot3pi_3_test.h5','main/data')

In [12]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/mnist_eqws_rotpi_3.h5", "w")
hf.create_dataset("train", data=mnist_DHC_out_rotpi_3, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("test", data=mnist_DHC_out_rotpi_3_test, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [13]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/mnist_eqws_rot2pi_3.h5", "w")
hf.create_dataset("train", data=mnist_DHC_out_rot2pi_3, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("test", data=mnist_DHC_out_rot2pi_3_test, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [14]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/mnist_eqws_rot3pi_3.h5", "w")
hf.create_dataset("train", data=mnist_DHC_out_rot3pi_3, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("test", data=mnist_DHC_out_rot3pi_3_test, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [3]:
mnist_DHC_train_LanRot = hd5_open('../from_cannon/2021_03_28/mnist_DHC_train_ang_LanRotResize.h5','data')
mnist_DHC_test_LanRot = hd5_open('../from_cannon/2021_03_28/mnist_DHC_test_ang_LanRotResize.h5','data')

In [4]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/mnist_eqws_Lanczos.h5", "w")
hf.create_dataset("train", data=mnist_DHC_train_LanRot, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("test", data=mnist_DHC_test_LanRot, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [15]:
mnist_train_y = hd5_open('../scratch_AKS/data/mnist_train_y.h5','main/data')
mnist_test_y = hd5_open('../scratch_AKS/data/mnist_test_y.h5','main/data')

In [17]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/mnist_labels.h5", "w")
hf.create_dataset("train", data=mnist_train_y, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("test", data=mnist_test_y, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [18]:
prec_mnist_angles = hd5_open('../scratch_AKS/data/prec_mnist_angles_3.h5','data')
prec_mnist_angles_noiso = hd5_open('../scratch_AKS/data/prec_mnist_angles_noiso_3.h5','data')
prec_mnist_angles_LanRotResize = hd5_open('../scratch_AKS/data/prec_mnist_angles_3_lanczosRotResize.h5','data')

In [19]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/mnist_eqws_fluct.h5", "w")
hf.create_dataset("ISO", data=prec_mnist_angles, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("REG", data=prec_mnist_angles_noiso, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("ISO_Lanczos", data=prec_mnist_angles_LanRotResize, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [20]:
test_angles = hd5_open('../scratch_AKS/data/angles_listjl.h5','main/test_angles')
train_angles = hd5_open('../scratch_AKS/data/angles_listjl.h5','main/train_angles')

In [21]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/angles_listjl.h5", "w")
hf.create_dataset("train_angles", data=train_angles, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("test_angles", data=test_angles, chunks=True, compression="gzip", compression_opts=9)
hf.close()

Need to makes sure we release the files used to recreate the below

In [22]:
std_class = hd5_open('../from_cannon/2021_03_09/equiv_workup_L3.h5','std_class')
std_class_both = hd5_open('../from_cannon/2021_03_09/equiv_workup_L3.h5','std_class_both')
std_class_ref = hd5_open('../from_cannon/2021_03_09/equiv_workup_L3.h5','std_class_ref')
equiv_std = hd5_open('../from_cannon/2021_03_09/equiv_workup_L3.h5','equiv_std_full')

hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/equiv_workup_L3.h5", "w")
hf.create_dataset("std_class", data=std_class, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_both", data=std_class_both, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_ref", data=std_class_ref, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("equiv_std", data=equiv_std, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [23]:
std_class = hd5_open('../from_cannon/2021_03_09/equiv_workup_L4.h5','std_class')
std_class_both = hd5_open('../from_cannon/2021_03_09/equiv_workup_L4.h5','std_class_both')
std_class_ref = hd5_open('../from_cannon/2021_03_09/equiv_workup_L4.h5','std_class_ref')
equiv_std = hd5_open('../from_cannon/2021_03_09/equiv_workup_L4.h5','equiv_std_full')

hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/equiv_workup_L4.h5", "w")
hf.create_dataset("std_class", data=std_class, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_both", data=std_class_both, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_ref", data=std_class_ref, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("equiv_std", data=equiv_std, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [24]:
std_class = hd5_open('../from_cannon/2021_03_09/equiv_workup_L5.h5','std_class')
std_class_both = hd5_open('../from_cannon/2021_03_09/equiv_workup_L5.h5','std_class_both')
std_class_ref = hd5_open('../from_cannon/2021_03_09/equiv_workup_L5.h5','std_class_ref')
equiv_std = hd5_open('../from_cannon/2021_03_09/equiv_workup_L5.h5','equiv_std_full')

hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/equiv_workup_L5.h5", "w")
hf.create_dataset("std_class", data=std_class, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_both", data=std_class_both, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_ref", data=std_class_ref, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("equiv_std", data=equiv_std, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [25]:
std_class = hd5_open('../from_cannon/2021_03_09/equiv_workup_L6.h5','std_class')
std_class_both = hd5_open('../from_cannon/2021_03_09/equiv_workup_L6.h5','std_class_both')
std_class_ref = hd5_open('../from_cannon/2021_03_09/equiv_workup_L6.h5','std_class_ref')
equiv_std = hd5_open('../from_cannon/2021_03_09/equiv_workup_L6.h5','equiv_std_full')

hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/equiv_workup_L6.h5", "w")
hf.create_dataset("std_class", data=std_class, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_both", data=std_class_both, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_ref", data=std_class_ref, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("equiv_std", data=equiv_std, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [26]:
std_class = hd5_open('../from_cannon/2021_03_09/equiv_workup_L7.h5','std_class')
std_class_both = hd5_open('../from_cannon/2021_03_09/equiv_workup_L7.h5','std_class_both')
std_class_ref = hd5_open('../from_cannon/2021_03_09/equiv_workup_L7.h5','std_class_ref')
equiv_std = hd5_open('../from_cannon/2021_03_09/equiv_workup_L7.h5','equiv_std_full')

hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/equiv_workup_L7.h5", "w")
hf.create_dataset("std_class", data=std_class, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_both", data=std_class_both, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_ref", data=std_class_ref, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("equiv_std", data=equiv_std, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [27]:
std_class = hd5_open('../from_cannon/2021_03_09/equiv_workup_L8.h5','std_class')
std_class_both = hd5_open('../from_cannon/2021_03_09/equiv_workup_L8.h5','std_class_both')
std_class_ref = hd5_open('../from_cannon/2021_03_09/equiv_workup_L8.h5','std_class_ref')
equiv_std = hd5_open('../from_cannon/2021_03_09/equiv_workup_L8.h5','equiv_std_full')

hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/equiv_workup_L8.h5", "w")
hf.create_dataset("std_class", data=std_class, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_both", data=std_class_both, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("std_class_ref", data=std_class_ref, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("equiv_std", data=equiv_std, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [28]:
ext_y = hd5_open('../from_cannon/2021_03_09/equivFit.h5','ext_y')
real_y = hd5_open('../from_cannon/2021_03_09/equivFit.h5','real_y')
y_interp = hd5_open('../from_cannon/2021_03_09/equivFit.h5','y_interp')
x_interp = hd5_open('../from_cannon/2021_03_09/equivFit.h5','x_interp')
x_for_compare = hd5_open('../from_cannon/2021_03_09/equivFit.h5','x_for_compare')
shifted_coeff = hd5_open('../from_cannon/2021_03_09/equivFit.h5','shifted_coeff')

In [29]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/equivFit.h5", "w")
hf.create_dataset("ext_y", data=ext_y, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("real_y", data=real_y, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("y_interp", data=y_interp, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("x_interp", data=x_interp, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("x_for_compare", data=x_for_compare, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("shifted_coeff", data=shifted_coeff, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [30]:
drop_test = np.zeros((10,3,309))
for i in range(10):
    for j in range(3):
        if j==0:
            drop_test[i,j,:] = hd5_open('../../DHC/from_cannon/2021_03_17/drop_test_{}.h5'.format(i),'drop_indx')
        if j==1:
            drop_test[i,j,:] = hd5_open('../../DHC/from_cannon/2021_03_17/drop_test_{}.h5'.format(i),'prec_max')
        if j==2:
            drop_test[i,j,:] = hd5_open('../../DHC/from_cannon/2021_03_17/drop_test_{}.h5'.format(i),'train_prec')

In [32]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/drop_test.h5", "w")
hf.create_dataset("data", data=drop_test, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [33]:
mnist_DHC_out_sizetrain = hd5_open('../scratch_AKS/data/mnist_DHC_out_rot7pi_8.h5','main/data')
mnist_DHC_out_sizetest = hd5_open('../scratch_AKS/data/mnist_DHC_out_rot7pi_8_test.h5','main/data')

In [34]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/mnist_eqws_rot7pi_8.h5", "w")
hf.create_dataset("train", data=mnist_DHC_out_sizetrain, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("test", data=mnist_DHC_out_sizetest, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [4]:
import scipy.io as sio
matfilters = sio.loadmat('/Users/saydjari/Dropbox/GradSchool_AKS/Doug/PythonRepos/phaseharmonics-master/matlab/filters/bumpsteerableg1_fft2d_N256_J8_L8.mat')

In [5]:
fftphi = matfilters['filt_fftphi'].astype(np.complex_)
hatphi = np.stack((np.real(fftphi), np.imag(fftphi)), axis=-1)

fftpsi = matfilters['filt_fftpsi'].astype(np.complex_)
hatpsi = np.stack((np.real(fftpsi), np.imag(fftpsi)), axis=-1)

In [6]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/bumpsteerable.h5", "w")
hf.create_dataset("hatphi", data=hatphi, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("hatpsi", data=hatpsi, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [41]:
with open('../../IWST/FromCannon/2020_10_29/res_morlet_cfree_scat7_acom.p', 'rb') as input_file:
    res = pickle.load(input_file)
    
with open("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/res_morlet_cfree_scat7_acom.h5", 'wb') as output_file:
    pickle.dump(res, output_file)

In [42]:
mnist_DHC_out = hd5_open('../scratch_AKS/data/mnist_DHC_out.h5','main/data')
mnist_DHC_out_test = hd5_open('../scratch_AKS/data/mnist_DHC_out_test.h5','main/data')

In [43]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/mnist_eqws.h5", "w")
hf.create_dataset("train", data=mnist_DHC_out, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("test", data=mnist_DHC_out_test, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [20]:
with open('../../IWST/FromCannon/2020_11_09/RWST_MHD_rinvar_cumsum.p', 'rb') as input_file:
    RWST_MHD_rinvar_cumsum = np.array(pickle.load(input_file))
with open('../../IWST/FromCannon/2020_11_09/WST_MHD_rinvar_cumsum.p', 'rb') as input_file:
    WST_MHD_rinvar_cumsum = np.array(pickle.load(input_file))

In [22]:
dhc_mhd = hd5_open("../../MHDSearch/from_cannon/2021_04_06/MHD_DHC_c1_L8_wd2_omegaF_apodF_S2T_S2rF_S2fF.h5","data")

In [23]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/MHD_WST_cumsum.h5", "w")
hf.create_dataset("RWST", data=RWST_MHD_rinvar_cumsum, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("WST", data=WST_MHD_rinvar_cumsum, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("EqWS", data=dhc_mhd, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [13]:
rod_L8_wd1_N1 = hd5_open('../scratch_AKS/paper_data/rod_stable_L8_wd1_N1.h5','data')
rod_L8_wd2_N1 = hd5_open('../scratch_AKS/paper_data/rod_stable_L8_wd2_N1.h5','data')
rod_L8_wd4_N1 = hd5_open('../scratch_AKS/paper_data/rod_stable_L8_wd4_N1.h5','data')

rod_L8_wd2_N4 = hd5_open('../scratch_AKS/paper_data/rod_stable_L8_wd2_N4.h5','data')
rod_L8_wd2_N16 = hd5_open('../scratch_AKS/paper_data/rod_stable_L8_wd2_N16.h5','data')

rod_angles = hd5_open('../scratch_AKS/paper_data/rod_stable_L8_wd2_N1.h5','angles')

In [14]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/rod_stability.h5", "w")
hf.create_dataset("wd1_N1", data=rod_L8_wd1_N1, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("wd2_N1", data=rod_L8_wd2_N1, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("wd4_N1", data=rod_L8_wd4_N1, chunks=True, compression="gzip", compression_opts=9)

hf.create_dataset("wd2_N4", data=rod_L8_wd2_N4, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("wd2_N16", data=rod_L8_wd2_N16, chunks=True, compression="gzip", compression_opts=9)

hf.create_dataset("angles", data=rod_angles, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [15]:
mnist_DHC_out_train_L4s = hd5_open('../from_cannon/2021_03_09/mnist_DHC_train_eq_L4_s.h5','main/data')
mnist_DHC_out_test_L4s = hd5_open('../from_cannon/2021_03_09/mnist_DHC_test_eq_L4_s.h5','main/data')

mnist_DHC_out_train_L4s_angles = hd5_open('../from_cannon/2021_03_09/mnist_DHC_train_eq_L4_s.h5','main/angles')
mnist_DHC_out_test_L4s_angles = hd5_open('../from_cannon/2021_03_09/mnist_DHC_test_eq_L4_s.h5','main/angles')

In [19]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/mnist_eqws_L4s.h5", "w")
hf.create_dataset("train", data=mnist_DHC_out_train_L4s, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("test", data=mnist_DHC_out_test_L4s, chunks=True, compression="gzip", compression_opts=9)

hf.create_dataset("train_angles", data=mnist_DHC_out_train_L4s_angles, chunks=True, compression="gzip", compression_opts=9)
hf.create_dataset("test_angles", data=mnist_DHC_out_test_L4s_angles, chunks=True, compression="gzip", compression_opts=9)
hf.close()

In [24]:
rod_corner = hd5_open('/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/from_cannon/2021_04_03/corner_rod_S2.h5','main/coeffs')
rod_curve = hd5_open('/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/from_cannon/2021_04_03/curve_rod_S2.h5','main/coeffs')
disc_data = hd5_open('/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/from_cannon/2021_04_03/disc_data_log_S2.h5','main/coeffs')

In [25]:
hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/rod_spaceS2.h5", "w")
hf.create_dataset("coeffs", data=rod_corner, chunks=True, compression="gzip", compression_opts=9)
hf.close()

hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/curve_spaceS2.h5", "w")
hf.create_dataset("coeffs", data=rod_curve, chunks=True, compression="gzip", compression_opts=9)
hf.close()

hf = h5py.File("/Users/saydjari/Dropbox/GradSchool_AKS/Doug/ZenodoRepos/EqWS/disc_spaceS2.h5", "w")
hf.create_dataset("coeffs", data=disc_data, chunks=True, compression="gzip", compression_opts=9)
hf.close()